In [3]:
import tensorflow as tf
import tensorflow_ranking as tfr

In [4]:
! wget -O "/tmp/train.txt" "https://raw.githubusercontent.com/tensorflow/ranking/master/tensorflow_ranking/examples/data/train.txt"
! wget -O "/tmp/test.txt" "https://raw.githubusercontent.com/tensorflow/ranking/master/tensorflow_ranking/examples/data/test.txt"

--2020-04-30 14:30:01--  https://raw.githubusercontent.com/tensorflow/ranking/master/tensorflow_ranking/examples/data/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29788 (29K) [text/plain]
Saving to: ‘/tmp/train.txt’

/tmp/train.txt      100%[===================>]  29.09K  --.-KB/s    in 0.02s   

2020-04-30 14:30:01 (1.25 MB/s) - ‘/tmp/train.txt’ saved [29788/29788]

--2020-04-30 14:30:02--  https://raw.githubusercontent.com/tensorflow/ranking/master/tensorflow_ranking/examples/data/test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.124.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.124.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8959 (8.7K) [text/plain]
Saving to: ‘/tmp/test.txt’

/tm

In [16]:
_TRAIN_DATA_PATH = "/tmp/train.txt"
_TEST_DATA_PATH = "/tmp/test.txt"

_LOSS = "pairwise_logistic_loss"

_LIST_SIZE = 100

_NUM_FEATURES = 136

_BATCH_SIZE = 32
_HIDDEN_LAYER_DIMS = ["20", "10"]
_LEARNING_RATE = 0.05

In [17]:
def input_fn(path):
    train_dataset = tf.data.Dataset.from_generator(
        tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
        output_types=(
            {str(k):tf.TensorShape([_LIST_SIZE, 1])
                for k in range(1, _NUM_FEATURES+1)},
            tf.TensorShape([_LIST_SIZE])
        )
    )
    train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()

In [22]:
def example_feature_columns():
    feature_names = [
        "%d" % (i+1) for i in range(0, _NUM_FEATURES)
    ]
    return{
        name: tf.feature_column.numeric_column(
            name, shape=(1,), default_value=0.0) for name in feature_names
    }

def make_score_fn():
    
    def _score_fn(context_features, group_features, mode, params, config):
        del params
        del config
        
        example_input = [
            tf.layers.flatten(group_features[name])
            for name in sorted(example_feature_columns())
        ]
        input_layer = tf.concat(example_input, 1)
        
        cur_layer = input_layer
        for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
            cur_layer = tf.layers.dense(
                cur_layer,
                units=layer_width,
                activation="tanh")
            
        logits = tf.layers.dense(cur_layer, units=1)
        return logits
    return _score_fn

def eval_metric_fns():
    """Returns a dict from name to metric functions.

    This can be customized as follows. Care must be taken when handling padded
    lists.

    def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
    metric_fns["auc"] = _auc

    Returns:
    A dict mapping from metric name to a metric function with above signature.
    """
    metric_fns = {}
    metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [1, 3, 5, 10]
    })

    return metric_fns

def get_estimator():
    """Create a ranking estimator.

    Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.

    Returns:
    tf.learn `Estimator`.
    """
    def _train_op_fn(loss):
        """Defines train op used in ranking head."""
        return tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_global_step(),
            learning_rate=_LEARNING_RATE,
            optimizer="Adagrad")

    ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

    return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
      params=None)

In [23]:
ranker = get_estimator()

W0430 14:52:15.308872 4411520448 estimator.py:1822] Using temporary folder as model directory: /var/folders/9_/0ln03qwj0qsbcg_vlv6ybn_m0000gn/T/tmp0msrx_s9
W0430 14:52:15.311163 4411520448 model_fn.py:630] Estimator's model_fn (<function _make_model_fn.<locals>._model_fn at 0x13037f730>) includes params argument, but params are not passed to Estimator.


In [24]:
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)

W0430 14:52:22.182719 4411520448 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0430 14:52:22.183812 4411520448 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow_core/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


AttributeError: module 'tensorflow_ranking.python.data' has no attribute 'libsvm_generator'

In [15]:
ranker.evaluate(input_fn=lambda: input_fn(_TEST_DATA_PATH), steps=100)

NameError: name 'ranker' is not defined